In [290]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [291]:
pd.set_option('display.max_columns', None)

In [292]:
df = pd.read_csv('retail_data.csv')

In [293]:
df.head(3)

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment,Date,Year,Month,Time,Total_Purchases,Amount,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788.000,37249.000,Michelle Harrington,Ebony39@gmail.com,1414786801.000,3959 Amanda Burgs,Dortmund,Berlin,77985.000,Germany,21.000,Male,Low,Regular,9/18/2023,2023.000,September,22:03:55,3.000,108.029,324.086,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.000,Cycling shorts
1,2174773.000,69749.000,Kelsey Hill,Mark36@gmail.com,6852899987.000,82072 Dawn Centers,Nottingham,England,99071.000,UK,19.000,Female,Low,Premium,12/31/2023,2023.000,December,8:42:04,2.000,403.354,806.708,Electronics,Samsung,Tablet,Excellent,Standard,Credit Card,Processing,4.000,Lenovo Tab
2,6679610.000,30192.000,Scott Jensen,Shane85@gmail.com,8362160449.000,4133 Young Canyon,Geelong,New South Wales,75929.000,Australia,48.000,Male,Low,Regular,4/26/2023,2023.000,April,4:06:29,3.000,354.478,1063.433,Books,Penguin Books,Children's,Average,Same-Day,Credit Card,Processing,2.000,Sports equipment


In [294]:
df.shape

(302010, 30)

In [295]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302010 entries, 0 to 302009
Data columns (total 30 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Transaction_ID    301677 non-null  float64
 1   Customer_ID       301702 non-null  float64
 2   Name              301628 non-null  object 
 3   Email             301663 non-null  object 
 4   Phone             301648 non-null  float64
 5   Address           301695 non-null  object 
 6   City              301762 non-null  object 
 7   State             301729 non-null  object 
 8   Zipcode           301670 non-null  float64
 9   Country           301739 non-null  object 
 10  Age               301837 non-null  float64
 11  Gender            301693 non-null  object 
 12  Income            301720 non-null  object 
 13  Customer_Segment  301795 non-null  object 
 14  Date              301651 non-null  object 
 15  Year              301660 non-null  float64
 16  Month             30

In [296]:
df.isnull().sum()

Transaction_ID      333
Customer_ID         308
Name                382
Email               347
Phone               362
Address             315
City                248
State               281
Zipcode             340
Country             271
Age                 173
Gender              317
Income              290
Customer_Segment    215
Date                359
Year                350
Month               273
Time                350
Total_Purchases     361
Amount              357
Total_Amount        350
Product_Category    283
Product_Brand       281
Product_Type          0
Feedback            184
Shipping_Method     337
Payment_Method      297
Order_Status        235
Ratings             184
products              0
dtype: int64

In [297]:
df = df.dropna()

In [298]:
df.isnull().sum()

Transaction_ID      0
Customer_ID         0
Name                0
Email               0
Phone               0
Address             0
City                0
State               0
Zipcode             0
Country             0
Age                 0
Gender              0
Income              0
Customer_Segment    0
Date                0
Year                0
Month               0
Time                0
Total_Purchases     0
Amount              0
Total_Amount        0
Product_Category    0
Product_Brand       0
Product_Type        0
Feedback            0
Shipping_Method     0
Payment_Method      0
Order_Status        0
Ratings             0
products            0
dtype: int64

In [299]:
# prompt: set the floats in pandas to 2 decimals

pd.set_option('display.float_format', '{:.2f}'.format)
df.head(3)


,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment,Date,Year,Month,Time,Total_Purchases,Amount,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788.00,37249.00,Michelle Harrington,Ebony39@gmail.com,1414786801.00,3959 Amanda Burgs,Dortmund,Berlin,77985.00,Germany,21.00,Male,Low,Regular,9/18/2023,2023.00,September,22:03:55,3.00,108.03,324.09,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.00,Cycling shorts
1,2174773.00,69749.00,Kelsey Hill,Mark36@gmail.com,6852899987.00,82072 Dawn Centers,Nottingham,England,99071.00,UK,19.00,Female,Low,Premium,12/31/2023,2023.00,December,8:42:04,2.00,403.35,806.71,Electronics,Samsung,Tablet,Excellent,Standard,Credit Card,Processing,4.00,Lenovo Tab
2,6679610.00,30192.00,Scott Jensen,Shane85@gmail.com,8362160449.00,4133 Young Canyon,Geelong,New South Wales,75929.00,Australia,48.00,Male,Low,Regular,4/26/2023,2023.00,April,4:06:29,3.00,354.48,1063.43,Books,Penguin Books,Children's,Average,Same-Day,Credit Card,Processing,2.00,Sports equipment


In [300]:
df.describe()

,Transaction_ID,Customer_ID,Phone,Zipcode,Age,Year,Total_Purchases,Amount,Total_Amount,Ratings
count,293911.00,293911.00,293911.00,293911.00,293911.00,293911.00,293911.00,293911.00,293911.00,293911.00
mean,5493725.59,55013.40,5500607077.13,50288.38,35.47,2023.17,5.36,255.15,1367.69,3.16
std,2596085.94,26009.44,2596111174.73,28976.61,15.02,0.37,2.87,141.39,1128.90,1.32
min,1000007.00,10000.00,1000049414.00,501.00,18.00,2023.00,1.00,10.00,10.00,1.00
25%,3245885.50,32470.00,3253497020.00,25408.00,22.00,2023.00,3.00,132.84,438.85,2.00
50%,5495879.00,55027.00,5504466161.00,50586.00,32.00,2023.00,5.00,255.46,1041.16,3.00
75%,7738197.00,77514.00,7749761285.50,75252.00,46.00,2023.00,8.00,377.64,2028.95,4.00
max,9999995.00,99999.00,9999996122.00,99949.00,70.00,2024.00,10.00,500.00,4999.63,5.00


In [301]:
dup = df.duplicated()
df[dup].index


Index([299759, 301362, 301486], dtype='int64')

In [302]:
df.drop(df.index[dup])

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment,Date,Year,Month,Time,Total_Purchases,Amount,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788.00,37249.00,Michelle Harrington,Ebony39@gmail.com,1414786801.00,3959 Amanda Burgs,Dortmund,Berlin,77985.00,Germany,21.00,Male,Low,Regular,9/18/2023,2023.00,September,22:03:55,3.00,108.03,324.09,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.00,Cycling shorts
1,2174773.00,69749.00,Kelsey Hill,Mark36@gmail.com,6852899987.00,82072 Dawn Centers,Nottingham,England,99071.00,UK,19.00,Female,Low,Premium,12/31/2023,2023.00,December,8:42:04,2.00,403.35,806.71,Electronics,Samsung,Tablet,Excellent,Standard,Credit Card,Processing,4.00,Lenovo Tab
2,6679610.00,30192.00,Scott Jensen,Shane85@gmail.com,8362160449.00,4133 Young Canyon,Geelong,New South Wales,75929.00,Australia,48.00,Male,Low,Regular,4/26/2023,2023.00,April,4:06:29,3.00,354.48,1063.43,Books,Penguin Books,Children's,Average,Same-Day,Credit Card,Processing,2.00,Sports equipment
3,7232460.00,62101.00,Joseph Miller,Mary34@gmail.com,2776751724.00,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420.00,Canada,56.00,Male,High,Premium,05-08-23,2023.00,May,14:55:17,7.00,352.41,2466.85,Home Decor,Home Depot,Tools,Excellent,Standard,PayPal,Processing,4.00,Utility knife
4,4983775.00,27901.00,Debra Coleman,Charles30@gmail.com,9098267635.00,5813 Lori Ports Suite 269,Bristol,England,48704.00,UK,22.00,Male,Low,Premium,01-10-24,2024.00,January,16:54:07,2.00,124.28,248.55,Grocery,Nestle,Chocolate,Bad,Standard,Cash,Shipped,1.00,Chocolate cookies
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302005,4246475.00,12104.00,Meagan Ellis,Courtney60@gmail.com,7466353743.00,389 Todd Path Apt. 159,Townsville,New South Wales,4567.00,Australia,31.00,Male,Medium,Regular,1/20/2024,2024.00,January,23:40:29,5.00,194.79,973.96,Books,Penguin Books,Fiction,Bad,Same-Day,Cash,Processing,1.00,Historical fiction
302006,1197603.00,69772.00,Mathew Beck,Jennifer71@gmail.com,5754304957.00,52809 Mark Forges,Hanover,Berlin,16852.00,Germany,35.00,Female,Low,New,12/28/2023,2023.00,December,2:55:45,1.00,285.14,285.14,Electronics,Apple,Laptop,Excellent,Same-Day,Cash,Processing,5.00,LG Gram
302007,7743242.00,28449.00,Daniel Lee,Christopher100@gmail.com,9382530370.00,407 Aaron Crossing Suite 495,Brighton,England,88038.00,UK,41.00,Male,Low,Premium,2/27/2024,2024.00,February,2:43:49,3.00,60.70,182.11,Clothing,Adidas,Jacket,Average,Express,Cash,Shipped,2.00,Parka
302008,9301950.00,45477.00,Patrick Wilson,Rebecca65@gmail.com,9373222023.00,3204 Baird Port,Halifax,Ontario,67608.00,Canada,41.00,Male,Medium,New,09-03-23,2023.00,September,11:20:31,1.00,120.83,120.83,Home Decor,IKEA,Furniture,Good,Standard,Cash,Shipped,4.00,TV stand


In [303]:
df[df['Name']=='Christine Kim']

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment,Date,Year,Month,Time,Total_Purchases,Amount,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
292839,4476510.00,20103.00,Christine Kim,James11@gmail.com,2383777672.00,8176 Randy Squares Apt. 772,Kitchener,Ontario,7099.00,Canada,54.00,Female,Low,Regular,12-10-23,2023.00,December,1:09:09,7.00,134.37,940.62,Clothing,Adidas,T-shirt,Bad,Express,Credit Card,Processing,1.00,Off-the-shoulder tee
299759,4476510.00,20103.00,Christine Kim,James11@gmail.com,2383777672.00,8176 Randy Squares Apt. 772,Kitchener,Ontario,7099.00,Canada,54.00,Female,Low,Regular,12-10-23,2023.00,December,1:09:09,7.00,134.37,940.62,Clothing,Adidas,T-shirt,Bad,Express,Credit Card,Processing,1.00,Off-the-shoulder tee


In [304]:
df = df.drop_duplicates()

In [332]:
df['Transaction_ID'].nunique()

287005

In [333]:
df['Transaction_ID'].value_counts()

Transaction_ID
6145934.00    4
9918529.00    3
6592672.00    3
1858467.00    3
2452608.00    3
             ..
4487444.00    1
6826016.00    1
2050516.00    1
6943631.00    1
9099074.00    1
Name: count, Length: 287005, dtype: int64

In [334]:
df[df['Transaction_ID']==6145934]

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment,Date,Year,Month,Time,Total_Purchases,Amount,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
140676,6145934.00,76353.00,Brian Alvarado,Richard77@gmail.com,5080492339.00,62419 Jonathan Orchard Suite 916,Hamilton,Ontario,89383.00,Canada,26.00,Male,Medium,Regular,11/29/2023,2023.00,January,15:48:29,3.00,146.11,438.34,Electronics,Whirepool,Fridge,Good,Express,Credit Card,Delivered,3.00,French door refrigerator
194626,6145934.00,23433.00,Joel Perry,Tiffany74@gmail.com,4748727451.00,572 Hunt Path,Bochum,Berlin,67691.00,Germany,22.00,Female,Low,Regular,3/24/2023,2023.00,March,10:38:14,1.00,180.74,180.74,Home Decor,Bed Bath & Beyond,Bedding,Bad,Same-Day,Credit Card,Delivered,1.00,Pillowcase set
300648,6145934.00,76353.00,Brian Alvarado,Richard77@gmail.com,5080492339.00,62419 Jonathan Orchard Suite 916,Hamilton,Ontario,89383.00,Canada,56.00,Male,Medium,Premium,11/29/2023,2023.00,November,15:48:29,3.00,146.11,438.34,Clothing,Zara,Dress,Good,Express,Cash,Pending,4.00,Casual dress
301371,6145934.00,23433.00,Joel Perry,Tiffany74@gmail.com,4748727451.00,572 Hunt Path,Bochum,Berlin,67691.00,Germany,22.00,Female,Low,Regular,3/24/2023,2023.00,March,10:38:14,1.00,180.74,180.74,Home Decor,Bed Bath & Beyond,Bedding,Bad,Same-Day,Cash,Delivered,1.00,Throw pillow


In [349]:
duplicate_transaction_ids = df['Transaction_ID'].value_counts()[lambda x: x > 1].index
duplicate_indexes = df[df['Transaction_ID'].isin(duplicate_transaction_ids)]
df = df.drop(duplicate_indexes)

In [352]:
df[df['Transaction_ID'].isin(duplicate_transaction_ids)]

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment,Date,Year,Month,Time,Total_Purchases,Amount,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788.00,37249.00,Michelle Harrington,Ebony39@gmail.com,1414786801.00,3959 Amanda Burgs,Dortmund,Berlin,77985.00,Germany,21.00,Male,Low,Regular,9/18/2023,2023.00,September,22:03:55,3.00,108.03,324.09,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.00,Cycling shorts
32,8973612.00,65701.00,Christopher Lopez,Derrick51@gmail.com,9286274243.00,57136 Martha Common Apt. 654,Portsmouth,England,11531.00,UK,66.00,Male,Medium,Regular,5/22/2023,2023.00,May,11:24:20,7.00,344.68,2412.75,Home Decor,IKEA,Decorations,Excellent,Same-Day,Debit Card,Processing,5.00,Curtains
42,3902930.00,77572.00,Jose Spencer,Amber67@gmail.com,8282194753.00,02936 Roy Village,Portsmouth,England,98771.00,UK,68.00,Female,High,Regular,10-06-23,2023.00,October,4:59:02,6.00,334.17,2004.99,Electronics,Apple,Laptop,Bad,Same-Day,Debit Card,Shipped,1.00,Microsoft Surface Laptop
72,9534452.00,20500.00,Matthew Hicks,Kara43@gmail.com,1568421682.00,51720 Rachael Spurs Suite 283,Portsmouth,England,94447.00,UK,61.00,Female,Medium,Regular,06-06-23,2023.00,June,8:43:39,4.00,201.95,807.81,Clothing,Zara,Dress,Average,Standard,PayPal,Processing,2.00,Maxi dress
93,1874788.00,74649.00,Courtney Perry,Tina32@gmail.com,7402421885.00,571 Samuel Wall Apt. 895,Portsmouth,England,24895.00,UK,18.00,Male,High,Regular,4/19/2023,2023.00,April,22:26:07,5.00,289.12,1445.58,Grocery,Nestle,Chocolate,Average,Express,Debit Card,Delivered,2.00,Chocolate-covered fruits
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302005,4246475.00,12104.00,Meagan Ellis,Courtney60@gmail.com,7466353743.00,389 Todd Path Apt. 159,Townsville,New South Wales,4567.00,Australia,31.00,Male,Medium,Regular,1/20/2024,2024.00,January,23:40:29,5.00,194.79,973.96,Books,Penguin Books,Fiction,Bad,Same-Day,Cash,Processing,1.00,Historical fiction
302006,1197603.00,69772.00,Mathew Beck,Jennifer71@gmail.com,5754304957.00,52809 Mark Forges,Hanover,Berlin,16852.00,Germany,35.00,Female,Low,New,12/28/2023,2023.00,December,2:55:45,1.00,285.14,285.14,Electronics,Apple,Laptop,Excellent,Same-Day,Cash,Processing,5.00,LG Gram
302007,7743242.00,28449.00,Daniel Lee,Christopher100@gmail.com,9382530370.00,407 Aaron Crossing Suite 495,Brighton,England,88038.00,UK,41.00,Male,Low,Premium,2/27/2024,2024.00,February,2:43:49,3.00,60.70,182.11,Clothing,Adidas,Jacket,Average,Express,Cash,Shipped,2.00,Parka
302008,9301950.00,45477.00,Patrick Wilson,Rebecca65@gmail.com,9373222023.00,3204 Baird Port,Halifax,Ontario,67608.00,Canada,41.00,Male,Medium,New,09-03-23,2023.00,September,11:20:31,1.00,120.83,120.83,Home Decor,IKEA,Furniture,Good,Standard,Cash,Shipped,4.00,TV stand


In [288]:
df['Transaction_ID'].value_counts()

Transaction_ID
2174773.000    1
2416371.000    1
5736738.000    1
5060644.000    1
2195548.000    1
              ..
3897203.000    1
2970871.000    1
3374752.000    1
9948479.000    1
9099074.000    1
Name: count, Length: 280216, dtype: int64

In [289]:
df = df.reset_index(drop=True)